<a href="https://colab.research.google.com/github/Colsai/603_Project_Files/blob/main/data_preparation_eda/LDA_vis_initial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workplan Topic Modeling
## Initial Test of LDA Modeling
- Looking through the current OIG work projects and group them using LDA.

In [34]:
#Pip install
!pip install pyLDAvis
!python -m spacy download en_core_web_md -qq
from IPython.display import clear_output
clear_output()

In [35]:
###########################
# Packages                #
###########################
import pandas as pd
import sklearn
import nltk
import spacy
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis.gensim_models
import en_core_web_md
import gensim
import random
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

# Set options for specific packages
nltk.download(['punkt', 
               'stopwords'])

# Visualise inside a notebook
pyLDAvis.enable_notebook()

sns.set()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
###########################
# Load CSVs (Github)      #
###########################
workplan_df = pd.read_csv("https://github.com/Colsai/scott_data606/blob/main/Data_Sources/HHS_OIG_workplans.csv?raw=true")

#Drop dead column
workplan_df.drop(columns = 'Office of Evaluation and Inspections', 
                 inplace = True)

reports_df = pd.read_csv("https://github.com/Colsai/scott_data606/blob/main/Data_Sources/HHS_OIG_Reports.csv?raw=true")

## Topic Modeling for Workplan df
Look at topics of previous projects decided on

In [38]:
workplan_df.head()

,Announced or Revised,Agency,Title,Component,Report Number(s),Expected Issue Date (FY),Website_Link,Summary
0,Completed,Administration for Children and Families,States' Accuracy of Reporting TANF Spending In...,Office of Audit Services,A-02-17-02005; W-00-17-25100,2021,https://oig.hhs.gov/reports-and-publications/w...,The Temporary Assistance for Needy Families (T...
1,Completed,Administration for Children and Families,Head Start: Review of Single Audit Findings an...,Office of Audit Services,"A-02-16-02009, A-09-16-01004, A-06-17-07003;...",2018,https://oig.hhs.gov/reports-and-publications/w...,Effective for awards made on or after December...
2,Completed,Administration for Children and Families,Unaccompanied Children Program Grantee Reviews,Office of Audit Services,A-02-16-02013; A-04-16-03566; A-02-16-02007;...,2020,https://oig.hhs.gov/reports-and-publications/w...,"Under the Homeland Security Act of 2002, § 462..."
3,Nov-16,Administration for Children and Families,Recommendation Follow-Up: Office of Refugee Re...,Office of Evaluation and Inspections,OEI-09-16-00260,2017,https://oig.hhs.gov/reports-and-publications/w...,"Under the Homeland Security Act of 2002, § 462..."
4,October 2020,Administration for Children and Families,ACF Oversight of Guardian Ad Litem Requirement...,Office of Evaluation and Inspections,OEI-12-16-00120,2022,https://oig.hhs.gov/reports-and-publications/w...,As a condition of receiving Child Abuse Preven...


In [64]:
###########################
# Clean Data              #
###########################

from nltk.stem import RegexpStemmer

stopwords = nltk.corpus.stopwords.words('english')

#Add stopwords
stopwords.append(['on',
                  'or', 
                  'to', 
                  'a', 
                  'as', 
                  'of', 
                  'for',
                  'this', 
                  'by', 
                  's',
                  'oig'])

#Set tokenizer to additionally remove punctuation
tokenizer = RegexpTokenizer(r'\w+')

wp_init_srs = [paragraph.lower() for paragraph in workplan_df["Summary"]]

tokenized_sums = [[i for i in tokenizer.tokenize(sent) if i not in stopwords] 
                  for sent in wp_init_srs]

# Regex Stemmer (Other Stemmers didn't seem to work well)
Reg_stemmer = RegexpStemmer("ing$|s$|ies$")

tokenized_stemmed_sums = [[Reg_stemmer.stem(word) for word in sent] for sent in tokenized_sums]

# Set new column of tokenized summaries
workplan_df["summary_tokenized"] = tokenized_stemmed_sums

workplan_df["summary_token_num"] = [len(sent) for sent in tokenized_stemmed_sums]

In [65]:
dictionary = Dictionary(workplan_df["summary_tokenized"])

#Filter extreme token values (no tokens below 5, no tokens in over 60% of documents)
dictionary.filter_extremes(no_below=5, 
                           no_above=0.5, 
                           keep_n=1000)

In [66]:
corpus = [dictionary.doc2bow(doc) for doc in workplan_df["summary_tokenized"]]

In [67]:
###############################
# LDA Data: Normal LDA        #
###############################
lda_model = LdaMulticore(corpus=corpus, 
                         id2word=dictionary, 
                         iterations=100, 
                         num_topics=10,
                         workers = 8, 
                         passes=20)

#Visualize the model with pyLDAvis
lda_model_visualize = pyLDAvis.gensim_models.prepare(lda_model,
                                                     corpus = corpus, 
                                                     dictionary = dictionary)
lda_model_visualize

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.063101 -0.075015       1        1  17.454832
7     -0.150325  0.040657       2        1  16.785868
8      0.201431  0.117689       3        1  12.905851
5      0.018840  0.027198       4        1  11.495116
1      0.132883 -0.094847       5        1   9.059323
0     -0.096753 -0.072766       6        1   8.271259
4      0.105581 -0.172895       7        1   7.690791
3     -0.017519  0.016854       8        1   7.321502
9     -0.204852  0.051886       9        1   5.186864
6      0.073814  0.161239      10        1   3.828594, topic_info=         Term        Freq       Total Category  logprob  loglift
237  medicare  948.000000  948.000000  Default  30.0000  30.0000
163      drug  423.000000  423.000000  Default  29.0000  29.0000
170  medicaid  551.000000  551.000000  Default  28.0000  28.0000
82   children  206.000000  206.000000  Default  27.0000  27.0000
41      state  742.000000  742.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
213   payment   17.630258  641.149353  Topic10  -4.6333  -0.3310
409   patient   13.252841  217.668484  Topic10  -4.9187   0.4639
71     review   13.296962  429.602089  Topic10  -4.9154  -0.2127
218  provider   12.431444  314.461515  Topic10  -4.9827   0.0320
302      part   11.159628  349.363940  Topic10  -5.0906  -0.1811

[583 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
46        1  0.264628   000
46        2  0.048114   000
46        8  0.601428   000
46        9  0.048114   000
246       1  0.149659    04
...     ...       ...   ...
105       5  0.123962  work
105       6  0.016904  work
105       7  0.045077  work
105       8  0.039442  work
105       9  0.073250  work

[1794 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 8, 9, 6, 2, 1, 5, 4, 10, 7])

In [61]:
#corpus = random.randint(0,len(corpus))

n = random.randint(0,len(workplan_df))

print(workplan_df.iloc[n]['Summary'], end = '')

for index, score in sorted(lda_model[corpus[n]], key=lambda tup: -1*tup[1]):
    
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

Medicare makes the full Medicare Severity Diagnosis-Related Group (MS-DRG) payment to a hospital that discharges an inpatient beneficiary "to home."  Under the post-acute-care transfer policy, however, for certain qualifying MS-DRGs, Medicare pays a hospital that transfers an inpatient beneficiary to post-acute care a per diem rate for each day of the stay, not to exceed the full MS-DRG payment that would have been made if the inpatient beneficiary had been discharged to home.  A prior OIG review identified Medicare overpayments to hospitals that did not comply with Medicare's post-acute-care transfer policy (42 CFR § 412.4(c)).  We found that hospitals transferred patients to certain post-acute-care settings but improperly claimed the higher reimbursement associated with discharges "to home."  Specifically, these hospitals used incorrect patient discharge status codes on their claims by indicating that their patients were discharged "to home" rather than transferred to a post-acute-ca

In [44]:
'''
from gensim.test.utils import datapath

# Save model to disk.
temp_file = datapath("/content/drive/MyDrive/DATA_606/lda_model")
lda_model.save(temp_file)

# Load a potentially pretrained model from disk.
#lda = LdaModel.load(temp_file)
'''

'\nfrom gensim.test.utils import datapath\n\n# Save model to disk.\ntemp_file = datapath("/content/drive/MyDrive/DATA_606/lda_model")\nlda_model.save(temp_file)\n\n# Load a potentially pretrained model from disk.\n#lda = LdaModel.load(temp_file)\n'

## Coherence Model Score
Score Coherence of Model - u_mass coherence score

In [62]:
#LDA Model Coherence Score
cm = CoherenceModel(model=lda_model, 
                    corpus=corpus, 
                    coherence="u_mass")

coherence = cm.get_coherence()  # get coherence value

print(coherence)

-2.04482422221679


In [63]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.083*"state" + 0.080*"medicaid" + 0.032*"payment" + 0.028*"federal" + 0.024*"care" + 0.023*"provider" + 0.015*"managed" + 0.013*"mco" + 0.011*"program" + 0.011*"requirement"
Topic: 1 
Words: 0.105*"drug" + 0.030*"fda" + 0.028*"manufacturer" + 0.024*"price" + 0.022*"amp" + 0.021*"asp" + 0.021*"food" + 0.021*"oig" + 0.021*"rebate" + 0.017*"medicare"
Topic: 2 
Words: 0.029*"hh" + 0.026*"health" + 0.026*"fund" + 0.022*"federal" + 0.017*"program" + 0.015*"audit" + 0.014*"response" + 0.014*"grant" + 0.013*"covid" + 0.013*"19"
Topic: 3 
Words: 0.026*"drug" + 0.025*"medicaid" + 0.021*"part" + 0.021*"medicare" + 0.019*"information" + 0.018*"oig" + 0.015*"state" + 0.014*"prescription" + 0.014*"program" + 0.014*"system"
Topic: 4 
Words: 0.068*"medicare" + 0.037*"payment" + 0.029*"hospital" + 0.019*"claim" + 0.018*"part" + 0.015*"care" + 0.012*"review" + 0.012*"requirement" + 0.012*"inpatient" + 0.012*"physician"
Topic: 5 
Words: 0.044*"nih" + 0.035*"health" + 0.024*"hospice" + 0

In [47]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=8, id2word=dictionary, passes=20, workers=8)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [48]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.019*"charge" + 0.015*"adverse" + 0.014*"event" + 0.013*"durable" + 0.011*"competitive" + 0.010*"1862" + 0.010*"orthotics" + 0.010*"prosthetics" + 0.008*"events" + 0.007*"purchase"
Topic: 1 Word: 0.022*"opioid" + 0.013*"fda" + 0.012*"orr" + 0.011*"drug" + 0.010*"food" + 0.010*"overdose" + 0.010*"asp" + 0.010*"drugs" + 0.008*"opioids" + 0.008*"prices"
Topic: 2 Word: 0.030*"foster" + 0.028*"background" + 0.018*"checks" + 0.017*"check" + 0.015*"technology" + 0.014*"employees" + 0.012*"criminal" + 0.010*"open" + 0.010*"numbers" + 0.009*"ordered"
Topic: 3 Word: 0.026*"hospice" + 0.022*"inpatient" + 0.016*"snf" + 0.015*"hospital" + 0.015*"transfer" + 0.014*"acute" + 0.012*"rehabilitation" + 0.010*"nurs" + 0.010*"stays" + 0.010*"condition"
Topic: 4 Word: 0.016*"dual" + 0.013*"drug" + 0.013*"overpayments" + 0.012*"sponsors" + 0.011*"part" + 0.011*"tools" + 0.010*"investigations" + 0.009*"account" + 0.009*"circular" + 0.009*"eligible"
Topic: 5 Word: 0.038*"mfcus" + 0.019*"mfcu" 

In [49]:
#corpus = random.randint(0,len(corpus))
n = 300
print(workplan_df.iloc[n]['Summary'], end = '')
for index, score in sorted(lda_model_tfidf[corpus[n]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

Opioid abuse and overdose deaths are at crisis levels in the United States, with approximately 49,000 Americans dying from opioids in 2017, an increase from more than 42,000 in 2016.   Consistent with previous OIG work in Medicare Part D and Medicaid, we will determine the extent to which beneficiaries are receiving extreme amounts of opioids through Indian Health Service (IHS), as well as IHS-employed prescribers and IHS-run pharmacies that have questionable prescribing or dispensing patterns.  This review will also determine how IHS prevents and detects opioid misuse or abuse, as well as how it enforces its opioid-related policies.
Score: 0.49678805470466614	 
Topic: 0.007*"medicaid" + 0.007*"medicare" + 0.006*"states" + 0.006*"care" + 0.005*"payments" + 0.005*"state" + 0.005*"health" + 0.005*"hhs" + 0.005*"cms" + 0.005*"federal"

Score: 0.4890599548816681	 
Topic: 0.022*"opioid" + 0.013*"fda" + 0.012*"orr" + 0.011*"drug" + 0.010*"food" + 0.010*"overdose" + 0.010*"asp" + 0.010*"drugs

In [50]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.07518621726725709),
 (1, 0.06253140396811553),
 (2, 0.08544882780091993),
 (3, 0.28214455001226385),
 (4, 0.10062692112949877),
 (5, 0.09129012757877557),
 (6, 0.07448811407386711),
 (7, 0.14629078635614434),
 (8, 0.07448811407386711),
 (9, 0.06297136925463026),
 (10, 0.11219291773812469),
 (11, 0.08544882780091993),
 (12, 0.09554443733221289),
 (13, 0.06577876964303928),
 (14, 0.5191046288800474),
 (15, 0.13357142744310574),
 (16, 0.23945336621559066),
 (17, 0.08879516266117218),
 (18, 0.05134206592025718),
 (19, 0.07818117345121553),
 (20, 0.12419742233806284),
 (21, 0.07186819751883873),
 (22, 0.0789866929559278),
 (23, 0.047882163340279726),
 (24, 0.10325631935743666),
 (25, 0.047144725326057874),
 (26, 0.06387412486391049),
 (27, 0.11219291773812469),
 (28, 0.07006214915886227),
 (29, 0.11219291773812469),
 (30, 0.09554443733221289),
 (31, 0.07249900679769361),
 (32, 0.057758158618575244),
 (33, 0.04838558882354207),
 (34, 0.10693989706187558),
 (35, 0.2188697239589579),
 (